In [1]:
import pandas as pd
import numpy as np

In [2]:
def normalize_types(df):
    for column in df.columns[df.dtypes == 'float64']:
        df[column] = df[column].astype(np.float32)

id = 1
X_train = pd.read_csv(f'./data/X_train_{id}.csv', dtype=np.float32).values
X_val = pd.read_csv(f'./data/X_val_{id}.csv', dtype=np.float32).values
X_test = pd.read_csv(f'./data/X_test_{id}.csv', dtype=np.float32).values

y_train = pd.read_csv(f'./data/y_train_{id}.csv', dtype=np.float32).values
y_val = pd.read_csv(f'./data/y_val_{id}.csv', dtype=np.float32).values
y_test = pd.read_csv(f'./data/y_test_{id}.csv', dtype=np.float32).values

ids_train = pd.read_csv(f'./data/ids_train_{id}.csv').values
ids_val = pd.read_csv(f'./data/ids_val_{id}.csv').values
ids_test = pd.read_csv(f'./data/ids_test_{id}.csv').values

buildings_train = pd.read_csv(f'./data/buildings_train_{id}.csv').values
buildings_val = pd.read_csv(f'./data/buildings_val_{id}.csv').values
buildings_test = pd.read_csv(f'./data/buildings_test_{id}.csv').values

print('train', buildings_train.shape, X_train.shape, y_train.shape, ids_train.shape)
print('val', buildings_val.shape, X_val.shape, y_val.shape, ids_val.shape)
print('test', buildings_test.shape, X_test.shape, y_test.shape, ids_test.shape)

train (302, 1) (2357111, 133) (2357111, 1) (2357111, 2)
val (101, 1) (771095, 133) (771095, 1) (771095, 2)
test (101, 1) (753623, 133) (753623, 1) (753623, 2)


In [3]:
def create_sequences(X, y, ids, buildings, time_steps=24):
    X_output = []
    y_output = []

    for building in buildings:
        X_values = X[ids[:,1] == building]
        y_values = y[ids[:,1] == building]

        for i in range(len(X_values) - time_steps + 1):
            X_output.append(X_values[i : (i + time_steps)])
            y_output.append(y_values[i])

    return np.stack(X_output), np.array(y_output)

time_step = 6
X_train_seq, y_train_seq = create_sequences(X_train, y_train, ids_train, buildings_train, time_step)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, ids_val, buildings_val, time_step)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, ids_test, buildings_test, time_step)

print('train', X_train_seq.shape, y_train_seq.shape)
print('val', X_val_seq.shape, y_val_seq.shape)
print('test', X_test_seq.shape, y_test_seq.shape)

train (2347011, 6, 133) (2347011, 1)
val (761900, 6, 133) (761900, 1)
test (744438, 6, 133) (744438, 1)


In [12]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import regularizers

In [5]:
import json

def save_results(model, history, test_eval, name):
    path = f'./5_hyperparameters_3_models/{name}'
    
    model.save(f'{path}/model')

    history_to_save = {}
    history_to_save['train'] = history
    history_to_save['test'] = test_eval

    with open(f"{path}/history.json", "w") as history_file:
        history_to_save_json = json.dumps(history_to_save, indent=4)
        history_file.write(history_to_save_json)

In [8]:
regs = [ None, 'l1', 'l2', 'l1l2' ]

activation, recurrent_activation, dropout, recurrent_dropout = 'sigmoid', 'tanh', 0.2, 0.1
cases = []
num = 0
for kernel_regularizer in regs:
    for recurrent_regularizer in regs:
        cases.append((num, activation, recurrent_activation, dropout, recurrent_dropout, kernel_regularizer, recurrent_regularizer))
        num += 1

cases_df = pd.DataFrame(cases, columns=['num', 'activation', 'recurrent_activation', 'dropout', 'recurrent_dropout', 'kernel_regularizer', 'recurrent_regularizer'])
cases_df

,num,activation,recurrent_activation,dropout,recurrent_dropout,kernel_regularizer,recurrent_regularizer
0,0,sigmoid,tanh,0.2,0.1,None,None
1,1,sigmoid,tanh,0.2,0.1,None,l1
2,2,sigmoid,tanh,0.2,0.1,None,l2
3,3,sigmoid,tanh,0.2,0.1,None,l1l2
4,4,sigmoid,tanh,0.2,0.1,l1,None
5,5,sigmoid,tanh,0.2,0.1,l1,l1
6,6,sigmoid,tanh,0.2,0.1,l1,l2
7,7,sigmoid,tanh,0.2,0.1,l1,l1l2
8,8,sigmoid,tanh,0.2,0.1,l2,None
9,9,sigmoid,tanh,0.2,0.1,l2,l1


In [14]:
for num, activation, recurrent_activation, dropout, recurrent_dropout, kernel_regularizer, recurrent_regularizer in cases[3:]:
    print(num, activation, recurrent_activation, dropout, recurrent_dropout, kernel_regularizer, recurrent_regularizer)

    kernel_regularizer_obj = None
    if kernel_regularizer == 'l1':
        kernel_regularizer_obj = regularizers.L1()
    elif kernel_regularizer == 'l2':
        kernel_regularizer_obj = regularizers.L2()
    elif kernel_regularizer == 'l1l2':
        kernel_regularizer_obj = regularizers.L1L2()
    
    recurrent_regularizer_obj = None
    if recurrent_regularizer == 'l1':
        recurrent_regularizer_obj = regularizers.L1()
    elif recurrent_regularizer == 'l2':
        recurrent_regularizer_obj = regularizers.L2()
    elif recurrent_regularizer == 'l1l2':
        recurrent_regularizer_obj = regularizers.L1L2()

    model = Sequential([
        layers.LSTM(
            32, 
            input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), 
            activation=activation,
            recurrent_activation=recurrent_activation,
            dropout=dropout, 
            recurrent_dropout=recurrent_dropout,
            kernel_regularizer=kernel_regularizer_obj,
            recurrent_regularizer=recurrent_regularizer_obj
        ),
        layers.Dense(1)
    ])
    model.compile(loss='mse', optimizer='adam', metrics=['mae', 'mape', 'mse'])

    model_history = model.fit(
        X_train_seq, 
        y_train_seq, 
        epochs=100, 
        batch_size=1024, 
        validation_data=(X_val_seq, y_val_seq), 
        shuffle=False,
        verbose=2,
        callbacks=[
            callbacks.EarlyStopping(monitor="val_loss", patience=10, mode="min", verbose=1, restore_best_weights=True)]
        )

    model_test_eval = model.evaluate(X_test_seq, y_test_seq, batch_size=1024)
    
    name = f'model-{num}'
    save_results(model, model_history.history, model_test_eval, name)

3 sigmoid tanh 0.2 0.1 None l1l2
Epoch 1/100
2293/2293 - 47s - loss: 6352.6992 - mae: 53.3653 - mape: 355.3243 - mse: 6352.6992 - val_loss: 4800.0298 - val_mae: 48.1181 - val_mape: 537.9233 - val_mse: 4800.0298 - 47s/epoch - 21ms/step
Epoch 2/100
2293/2293 - 45s - loss: 4625.4863 - mae: 46.3462 - mape: 402.6004 - mse: 4625.4863 - val_loss: 3760.1484 - val_mae: 40.4459 - val_mape: 341.5817 - val_mse: 3760.1484 - 45s/epoch - 20ms/step
Epoch 3/100
2293/2293 - 47s - loss: 3909.1816 - mae: 44.2188 - mape: 333.5950 - mse: 3909.1816 - val_loss: 3504.1907 - val_mae: 39.5792 - val_mape: 330.5952 - val_mse: 3504.1907 - 47s/epoch - 21ms/step
Epoch 4/100
2293/2293 - 47s - loss: 3672.2427 - mae: 43.4450 - mape: 321.5863 - mse: 3672.2427 - val_loss: 3355.1084 - val_mae: 39.3470 - val_mape: 336.6378 - val_mse: 3355.1084 - 47s/epoch - 20ms/step
Epoch 5/100
2293/2293 - 47s - loss: 3540.8535 - mae: 42.2524 - mape: 307.3528 - mse: 3540.8535 - val_loss: 3244.4353 - val_mae: 37.0190 - val_mape: 284.3622 - 

INFO:tensorflow:Assets written to: ./5_hyperparameters_3_models/model-3/model\assets


4 sigmoid tanh 0.2 0.1 l1 None
Epoch 1/100
2293/2293 - 51s - loss: 6293.8174 - mae: 53.1690 - mape: 366.2275 - mse: 6290.2217 - val_loss: 4831.0396 - val_mae: 48.8764 - val_mape: 575.1016 - val_mse: 4828.8574 - 51s/epoch - 22ms/step
Epoch 2/100
2293/2293 - 47s - loss: 4681.6665 - mae: 47.0888 - mape: 428.3603 - mse: 4675.6338 - val_loss: 3822.1606 - val_mae: 41.7916 - val_mape: 449.3416 - val_mse: 3812.6868 - 47s/epoch - 21ms/step
Epoch 3/100
2293/2293 - 47s - loss: 3912.9548 - mae: 44.1799 - mape: 342.5746 - mse: 3903.9951 - val_loss: 3454.3821 - val_mae: 39.5900 - val_mape: 367.8430 - val_mse: 3444.7910 - 47s/epoch - 20ms/step
Epoch 4/100
2293/2293 - 46s - loss: 3686.5432 - mae: 43.6209 - mape: 322.9709 - mse: 3676.5347 - val_loss: 3280.1506 - val_mae: 38.3898 - val_mape: 332.8085 - val_mse: 3268.9526 - 46s/epoch - 20ms/step
Epoch 5/100
2293/2293 - 46s - loss: 3524.0366 - mae: 42.4278 - mape: 299.0812 - mse: 3512.2156 - val_loss: 3185.8381 - val_mae: 35.3577 - val_mape: 204.8927 - va

INFO:tensorflow:Assets written to: ./5_hyperparameters_3_models/model-4/model\assets


5 sigmoid tanh 0.2 0.1 l1 l1
Epoch 1/100
2293/2293 - 48s - loss: 6302.0439 - mae: 53.1229 - mape: 358.0248 - mse: 6294.2910 - val_loss: 4829.1890 - val_mae: 48.4940 - val_mape: 551.3431 - val_mse: 4820.9336 - 48s/epoch - 21ms/step
Epoch 2/100
2293/2293 - 46s - loss: 4578.0825 - mae: 45.9988 - mape: 388.9713 - mse: 4566.9023 - val_loss: 4166.3994 - val_mae: 43.2956 - val_mape: 387.1135 - val_mse: 4153.7119 - 46s/epoch - 20ms/step
Epoch 3/100
2293/2293 - 46s - loss: 3851.0286 - mae: 43.7562 - mape: 365.3172 - mse: 3837.8499 - val_loss: 5213.2700 - val_mae: 47.6724 - val_mape: 305.8122 - val_mse: 5198.8760 - 46s/epoch - 20ms/step
Epoch 4/100
2293/2293 - 46s - loss: 3787.6348 - mae: 44.1134 - mape: 333.6133 - mse: 3772.5725 - val_loss: 3610.4836 - val_mae: 39.6698 - val_mape: 324.7452 - val_mse: 3594.4426 - 46s/epoch - 20ms/step
Epoch 5/100
2293/2293 - 46s - loss: 3633.0659 - mae: 42.9778 - mape: 327.9717 - mse: 3616.0732 - val_loss: 4190.3955 - val_mae: 41.6197 - val_mape: 206.2278 - val_

...

INFO:tensorflow:Assets written to: ./5_hyperparameters_3_models/model-13/model\assets


14 sigmoid tanh 0.2 0.1 l1l2 l2
Epoch 1/100
2293/2293 - 94s - loss: 6369.2607 - mae: 53.4103 - mape: 351.9267 - mse: 6368.6592 - val_loss: 4828.9390 - val_mae: 48.4616 - val_mape: 549.5345 - val_mse: 4828.4756 - 94s/epoch - 41ms/step
Epoch 2/100
2293/2293 - 90s - loss: 4646.8599 - mae: 47.0596 - mape: 450.6821 - mse: 4645.7485 - val_loss: 3902.6729 - val_mae: 41.2897 - val_mape: 361.9279 - val_mse: 3901.2246 - 90s/epoch - 39ms/step
Epoch 3/100
2293/2293 - 90s - loss: 3998.3962 - mae: 45.0045 - mape: 402.5344 - mse: 3996.8230 - val_loss: 3827.5483 - val_mae: 40.4924 - val_mape: 318.1877 - val_mse: 3825.8379 - 90s/epoch - 39ms/step
Epoch 4/100
2293/2293 - 89s - loss: 3796.0256 - mae: 44.3317 - mape: 323.7347 - mse: 3794.1602 - val_loss: 4163.9072 - val_mae: 41.4424 - val_mape: 280.6424 - val_mse: 4161.9917 - 89s/epoch - 39ms/step
Epoch 5/100
2293/2293 - 90s - loss: 3831.9641 - mae: 43.1970 - mape: 315.5282 - mse: 3829.8323 - val_loss: 3774.7622 - val_mae: 38.9677 - val_mape: 249.5286 - v

INFO:tensorflow:Assets written to: ./5_hyperparameters_3_models/model-14/model\assets


15 sigmoid tanh 0.2 0.1 l1l2 l1l2
Epoch 1/100
2293/2293 - 79s - loss: 6253.3706 - mae: 52.9753 - mape: 365.3383 - mse: 6253.3706 - val_loss: 4760.4463 - val_mae: 48.0570 - val_mape: 545.2001 - val_mse: 4760.4463 - 79s/epoch - 34ms/step
Epoch 2/100
2293/2293 - 75s - loss: 4582.9292 - mae: 46.0639 - mape: 378.8862 - mse: 4582.9292 - val_loss: 3740.9082 - val_mae: 40.4888 - val_mape: 352.0985 - val_mse: 3740.9082 - 75s/epoch - 33ms/step
Epoch 3/100
2293/2293 - 75s - loss: 3870.7031 - mae: 44.0499 - mape: 329.8671 - mse: 3870.7031 - val_loss: 3473.9697 - val_mae: 39.6839 - val_mape: 340.5815 - val_mse: 3473.9697 - 75s/epoch - 33ms/step
Epoch 4/100
2293/2293 - 73s - loss: 3712.1409 - mae: 43.3326 - mape: 310.2152 - mse: 3712.1409 - val_loss: 3211.7083 - val_mae: 37.4755 - val_mape: 296.7780 - val_mse: 3211.7083 - 73s/epoch - 32ms/step
Epoch 5/100
2293/2293 - 73s - loss: 3385.3174 - mae: 41.0372 - mape: 265.5164 - mse: 3385.3174 - val_loss: 3329.7466 - val_mae: 36.8026 - val_mape: 177.3911 -

INFO:tensorflow:Assets written to: ./5_hyperparameters_3_models/model-15/model\assets
